<a href="https://colab.research.google.com/github/KarinaKarina6/NLP_HWs/blob/HW_2/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers[torch] --upgrade
!pip install accelerate -U

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset, load_metric


Задача состоит в том, чтобы создать вопросно-ответную систему с использованием набора данных SQuAD.

План:

1.   Выбор модели.
2.   Загрузка и подготовка набор данных SQuAD.
3.   Дообучение модели.
4.   Измерение качества модели.


#### 1. Выбор модели

Для системы вопрос-ответ обычно используются модели на основе трансформеров, такие как BERT, RoBERTa и DistilBERT. Эти модели можно настроить под конкретную задачу ответа на вопрос. Учитывая популярность и производительность, я буду использовать модель "bert-base-uncased" из библиотеки Transformers Hugging Face.

In [ ]:
model_name = "bert-base-uncased"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#### 2. Загрузка и подготовка набор данных SQuAD

In [13]:
dataset = load_dataset("squad")
train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
validation_dataset = dataset["validation"].shuffle(seed=42).select(range(100))


#### 3. Дообучение модели

 Дообучение модели состоит ее обучение на наборе данных SQuAD, чтобы адаптировать ее к конкретной задаче ответа на вопросы. Нужно предварительно обработать данные, а затем обучить модель.

In [14]:
# Предобработка данных
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt",
    )
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        sequence_ids = inputs.sequence_ids(i)
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(tokenizer.model_max_length)
            end_positions.append(tokenizer.model_max_length)
        else:
            start_idx = context_start
            while start_idx < len(offset) and offset[start_idx][0] <= start_char:
                start_idx += 1
            end_idx = context_start
            while end_idx < len(offset) and offset[end_idx][1] <= end_char:
                end_idx += 1

            start_positions.append(start_idx - 1)
            end_positions.append(end_idx - 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
validation_dataset = validation_dataset.map(preprocess_function, batched=True, remove_columns=validation_dataset.column_names)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [15]:
# Определение training_arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
)

In [16]:
# Определение trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

In [ ]:
# Файн-тюнинг модели
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


Очень долго считается, не успело досчитаться(

#### 4. Измерение качества модели

Для измерения качетсва модели будут использованы показатели, такие как точное соответствие (EM) и показатель F1, которые являются стандартными для оценки вопросно-ответных систем.

In [ ]:
metric = load_metric("squad")


In [ ]:

def compute_metrics(pred):
    predictions, labels = pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Вычисление EM и F1
    results = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return results


In [ ]:
# Оценка модели
evaluation_results = trainer.evaluate(eval_dataset=validation_dataset, metric_key_prefix="eval")
print(evaluation_results)